# Recipe to automatize FT-ICR spectra phasing
a notebook to tune the algo, and see it running live...

check for **`# Parameters`**

In [ ]:
# load all python and interactive tools - should be run only once
from IPython.display import display, HTML, Markdown, Image
display(Markdown('## STARTING Environment...'))
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget
import spike
from spike.File import BrukerMS
from spike.Interactive import INTER as I
from spike.Interactive import FTICR_INTER as FI
from spike.Interactive import INTER_MS as IMS
from spike.Interactive.ipyfilechooser import FileChooser
print("\nInteractive module version,",I.__version__)
from datetime import datetime
print('Run date:', datetime.now().isoformat() )
I.initialize()
display(Markdown('## ...program is Ready'))
from importlib import reload  # this line is debugging help

# configurable items
mpl.rcParams['figure.figsize'] = (8,4)   # (X,Y) default figure size
I.Activate_Wheel = True                  # True/False wheel control in the graphic cells 


from spike.plugins.Peaks import peak_aggreg
import os
class FCC:
    pass
FC = FCC()

# Prepare
### load

In [ ]:
# Parameters
FC.selected = "/home/mad/Documents/DATA/DATA_res/FT-ICR/spectres MS complexes/polyA_ms_000001.d/fid"
FC.selected = "/home/mad/Documents/DATA/DATA/FT-ICR/MvA/Phase/histonepeptide_ms2_000003.d/fid"

FC.selected_path = os.path.dirname(FC.selected)
print('Reading file ',FC.selected)
d1 = BrukerMS.Import_1D(FC.selected)
d1.filename = FC.selected
d1.set_unit('sec').display(title=FC.selected_path+" transient")
d1

### compute modulus spectrum

In [ ]:
D1md = d1.copy() # copy the imported data-set to another object for processing
D1md.kaiser(6).zf(4).rfft().modulus() # kaiser(4) is an apodisation well adapted to FTICR, slightly more resolution than hamming(
D1md.set_unit('m/z')#.display(title=FC.selected_path)  # set to ppm unit - and display

### Find peaks and holes - and show - DON'T INCLUDE harmonics !

In [ ]:
# Parameters
# Region of interest - in m/z
lowmassOI = 400 #D1md.axis1.lowmass #472
highmassOI = D1md.axis1.highmass

# peak picking
noisethreshold = 10    # sigma
minimaldistance = 100  # in pixel

# baseline
baselinestrength = 2   # 1..4

# find peaks
D1md.set_unit('m/z').pp(autothresh=noisethreshold,  zoom=(highmassOI, lowmassOI))
D1md.centroid()
pks = peak_aggreg(D1md.peaks,minimaldistance)
print(len(pks),'stored peaks')

# find holes
from spike.plugins.bcorr import autopoints
import matplotlib.pyplot as plt
P1,P2 = IMS.firstguess(D1md)
xp = autopoints(D1md, int(baselinestrength*P2))

#show
D1md.set_unit("points").display(title=FC.selected_path)
plt.scatter(pks.pos, D1md.buffer[pks.pos.astype('int')], marker='x', c='b',label='peaks')
plt.scatter(xp, D1md.buffer[xp], marker='.', c='r', label='base points')
plt.legend()
plt.gca().set_xbound(D1md.axis1.mztoi(lowmassOI),D1md.axis1.mztoi(highmassOI))  # to zoom in ROI

In [ ]:
D1md.axis1.highmass, D1md.axis1.itomz(100000), D1md.axis1.lowmass, D1md.axis1.itomz(500000) # limite possible dans le spectre

Applied: phaseMS(70.5,  -1569.8, 16004.4, 0.00) + First Guess !

# 1st step
### compute phase-sensitive , and apply first guess

In [ ]:
D1 = d1.copy() # copy the imported data-set to another object for processing
D1.apod_sin(maxi=0.15).zf(4).rfft().set_unit('points')
P1,P2 = IMS.firstguess(D1)     # compute and apply firstguest
print("first guess:",P1,P2)
D1.phaseMS(0, P1, P2, 0.0)

In [ ]:
D1.display()

# 2nd step
### estimate P0 , P1 on each peaks

#### show priniple of phasing one little area

In [ ]:
# Parameters
# width around peak, in Dalton
width = 1

# the largest one
largest = np.argmax(pks.intens)
print("testing on peak",pks[largest].report())
center = pks[largest].pos

# define zone around in points
largmz = D1md.axis1.itomz(center)
oneDalton = (D1md.axis1.mztoi(largmz-1)-center)  
zone = [int(center-width*oneDalton), int(center+width*oneDalton)]
dd = spike.NPKData._NPKData(buffer=D1.get_buffer()[zone[0]:zone[1]]).display(label='before')
dd.apmin(first_order=False)
p0, p1 = dd.axis1.P0, dd.axis1.P1
dd.apmin(first_order=True, bcpoints=4)
p0, p1 = p0+dd.axis1.P0, p1+dd.axis1.P1
dd.bcorr(xpoints=100) #int(abs(baselinestrength*dd.axis1.P1)))
dd.display(new_fig=False, label='after')
plt.scatter(2*(center-zone[0]), pks[largest].intens)
print('P0, P1',p0, p1)

### do zone phasing around all peaks (kind of lengthy)

In [ ]:
# Parameters
width = 2  # in Dalton

def pkphase(D, Dm, pk, width=1, disp=False):
    "phase 1st order around a peak, +/- width around in m/z"
        # define zone around in points
    center = pk.pos
    largmz = Dm.axis1.itomz(center)
    oneDalton = (Dm.axis1.mztoi(largmz-1)-center)  
    zone = [int(center-width*oneDalton), int(center+width*oneDalton)]
    dd = spike.NPKData._NPKData(buffer=D.get_buffer()[zone[0]:zone[1]] )
    p0, p1 = 0, 0
    if disp:
        dd.display(label='avant')
    dd.apmin(first_order=False)
    p0, p1 = p0+dd.axis1.P0, p1+dd.axis1.P1
    dd.apmin(first_order=True, bcpoints=6, debug=False)
    p0, p1 = p0+dd.axis1.P0, p1+dd.axis1.P1
    if disp:
        plt.scatter(2*(center-zone[0]), pk.intens)
        dd.display(label='apres',new_fig=False)
    return (    p0, p1, zone )
# do over whole spectrum
pl0 = []
pl1 = []
xpk = []
for i in range(len(pks)):
    try:
        p0,p1, zone = pkphase(D1, D1md, pks[i], width=width, disp=False)
    except:
        continue
    if p1 !=0:
        pp1 = D1.size1*p1/(zone[1]-zone[0])
        pl0.append(p0)
        pl1.append(pp1)
        xpk.append(pks[i].pos)
pl1 = np.array(pl1)/360 # change P1 in turns
xpk = np.array(xpk)


### show P0 and P1 estimated over the whole spectrum

In [ ]:
plt.figure()
plt.scatter(xpk, pl0, marker='+',label='P0')
plt.scatter(xpk, pl1, marker='x',label='P1')
plt.legend()

### fit P1 to  a line using robust stat

In [ ]:
from scipy.optimize import minimize
def fitL1(xpeaks, p1list):
    def costL1(sol):
        "cost function implementing L1 norm"
        a,b = sol
        y = a*xpeaks+b
        return sum(abs(y-p1list))
    ini = (0,0)
    resL1 = minimize(costL1, ini)
    return resL1.x
pp2,pp1 = fitL1(xpk, pl1)
plt.plot(xpk,pp2*xpk+pp1, label='1st fit')
print(pp1,pp2)

### remove outliers and redo

In [ ]:
# Parameters
sigma_outliers = 1.0

res = (pp2*xpk+pp1-pl1)
res.mean(), res.std()
xpk2 = xpk[abs(res)<sigma_outliers*res.std()]
pl12 = pl1[abs(res)<sigma_outliers*res.std()]
print(len(xpk),len(xpk2))
pp2,pp1 = fitL1(xpk2, pl12)
plt.plot(xpk,pp2*xpk+pp1, label='2nd fit')
plt.scatter(xpk2, pl12, marker='+', label='2nd fit')
plt.legend()
print(pp1,pp2)

### as a check, apply this correction

In [ ]:
P2,pp2*D1.cpxsize1/4, P2+pp2*D1.cpxsize1/4

In [ ]:
PP1 = pp1
PP2 = pp2*D1.cpxsize1/4  # WHY 4 it should be 2 !!!
print(P1+PP1, P2+PP2)
D1.phaseMS(0, PP1, PP2, 0).display()


it's not done yet, errors remain

So we should evaluate the error here and either continue or loop back to **2nd step**

# 3rd step - Fine tune using 0th order.

### propagate correction to initial p0 values

In [ ]:
nxpk = xpk/D1.cpxsize1
newpl0 = (np.array(pl0)/360 - nxpk*(PP1 + PP2*nxpk)+0.5)%1  # changed unit to turns for homogeneity

In [ ]:
plt.figure()
plt.scatter(xpk,newpl0,label='observed phase')
plt.ylabel('P0')
plt.legend()
plt.title('observed phase after 2nd step');

In [ ]:
# unfold phase v2
skip = newpl0[1:]-newpl0[:-1]
#plt.scatter(xpk[1:],skip)
for i in range(1,len(xpk)):
    if skip[i-1]>0.5:
        newpl0[i:] -= 1
    if skip[i-1]<-0.5:
        newpl0[i:] += 1
plt.scatter(xpk, newpl0, marker='+')

In [ ]:
# unfold phase v1
skip = newpl0[1:]-newpl0[:-1]
#plt.scatter(xpk[1:],skip)
for i in range(1,len(xpk)):
    if skip[i-1]>0.5:
        newpl0[i:] -= 1
    if skip[i-1]<-0.5:
        newpl0[i:] += 1
plt.scatter(xpk, newpl0, marker='+')

# and fit a 2nd order polynomial to it

In [ ]:
# first
final = np.polyfit(xpk, newpl0, 2)
plt.plot(xpk, np.polyval(final,xpk),label='1st fit')
# reject outliers
res = newpl0-np.polyval(final,xpk)
xpk2 = xpk[abs(res)<2*res.std()]
new2 = newpl0[abs(res)<2*res.std()]
# redo
final = np.polyfit(xpk2, new2, 2)
plt.plot(xpk, np.polyval(final,xpk),label='2nd fit')
plt.scatter(xpk2, new2, marker='+', label='2nd fit')
plt.legend()
plt.title('2nd order fit of the residual phase, after unfolding')
print('P0 (degrees), P1, P2:',final[2]*360, final[1], final[0]*D1.cpxsize1)


In [ ]:
newnew = (np.array(pl0)/360 + np.polyval(final,xpk))%1
plt.figure()
plt.scatter(xpk, newnew)


In [ ]:
print (360*final[2], P1+PP1+final[1], P2+PP2+final[0]*D1.cpxsize1)
D1 = d1.copy() # copy the imported data-set to another object for processing
D1.apod_sin(maxi=0.15).zf(4).rfft()
D1.phaseMS(-360*final[2], P1+PP1-final[1], P2+PP2+final[0]*D1.cpxsize1, 0).set_unit('m/z').display()


# Left overs - do not use

In [ ]:
x = []
y = []
for i in range(-8,9,2):
    x.append(dd[int(round(2*(center-zone[0])+i))])
    y.append(dd[int(round(2*(center-zone[0])+i+1))])
plt.figure()
plt.scatter(x,y)
plt.plot(0,0,'+')
np.arctan2(sum(y),sum(x))*180/np.pi, sum(y), sum(x)

In [ ]:
def pkdispa(data, pk, extend):
    x = []
    y = []
    for i in range(-2*(extend//2),2*(extend//2)+1,2):
        x.append(data[2*int(round(pk.pos+i))])
        y.append(data[2*int(round(pk.pos+i))+1])
    return np.arctan2(sum(y),sum(x))*180/np.pi    
from spike.plugins.Peaks import peak_aggreg

zone = [int(center-10*oneDalton), int(center+10*oneDalton)]

dd = spike.NPKData._NPKData(buffer=D1.get_buffer()[zone[0]:zone[1]]).phase(90-160,-800).display(label='avant')
#dd.apmin(first_order=False)
#dd.apmin(first_order=True, bcpoints=20)
dd.display(new_fig=False, label='apres')

ddm = dd.copy().modulus().pp(autothresh=10)
dd.peaks = peak_aggreg(ddm.peaks, 10)
dd.display_peaks()

In [ ]:
pkph = []
for pk in dd.peaks:
    pkph.append(pkphase(dd,pk,10))
plt.figure()
plt.scatter(dd.peaks.pos,pkph)

In [ ]:
p0, p1 = dd.axis1.P0, dd.axis1.P1
p0,p1

In [ ]:
p0,p1

In [ ]:
zone = [int(center-5*oneDalton), int(center+5*oneDalton)]
dd = spike.NPKData._NPKData(buffer=D1.get_buffer()[zone[0]:zone[1]]).phase(5*p0,5*p1).display(label='avant')
#dd.apmin(first_order=False)
#dd.apmin(first_order=True,bcpoints=10)
dd.display(new_fig=False)
dd.bcorr(xpoints=100)
dd.display(new_fig=False, label='après')

In [ ]:
dd.axis1.P0, dd.axis1.P1
